<a href="https://colab.research.google.com/github/nvvishnu/CS6910-Assignments/blob/main/Assignment%203/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import csv
import tensorflow as tf
from tensorflow import keras
from keras.utils.vis_utils import plot_model
!pip install wandb
import wandb
from wandb.keras import WandbCallback 
import pandas as pd
!pip install dataframe-image
import dataframe_image as dfi

  Using cached https://files.pythonhosted.org/packages/98/5f/45439b4767334b868e1c8c35b1b0ba3747d8c21be77b79f09eed7aa3c72b/wandb-0.10.30-py2.py3-none-any.whl
     |████████████████████████████████| 133kB 2.9MB/s 
  Using cached https://files.pythonhosted.org/packages/fd/01/ff260a18caaf4457eb028c96eeb405c4a230ca06c8ec9c1379f813caa52e/configparser-5.0.2-py3-none-any.whl
     |████████████████████████████████| 174kB 13.9MB/s 
  Using cached https://files.pythonhosted.org/packages/32/c8/564be4d12629b912ea431f1a50eb8b3b9d00f1a0b1ceff17f266be190007/subprocess32-3.5.4.tar.gz
  Using cached https://files.pythonhosted.org/packages/f5/e8/f6bd1eee09314e7e6dee49cbe2c5e22314ccdb38db16c9fc72d2fa80d054/docker_pycreds-0.4.0-py2.py3-none-any.whl
     |████████████████████████████████| 71kB 8.0MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=db33978bf90d8931b846cb60c837ef9b2c7bc60b55bf8353ee7d5d1254d77e3c
  Stored in directory: /root/.cache/pip/wheels/0b/04

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Train data preprocessing

In [ ]:
datapath = "/content/drive/MyDrive/Assignment3/hi/lexicons/hi.translit.sampled.train.tsv" #add path here after mounting drive 
input_texts = []
target_texts = []
expected_texts = []
input_characters = set([' '])
target_characters = set([' '])
f = open(datapath)
lines = csv.reader(f,delimiter='\t')
 
for row in lines:
    input_text = row[1]
    target_text = row[0]
    #adding '\t' as start and '\n' as stop
    expected_texts.append(target_text)
    target_text = "\t"+target_text+"\n"
    
 
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
 
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
 
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)
 
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
 
encoder_input_data = np.zeros((len(input_texts),max_encoder_seq_length),dtype='float32')
decoder_input_data = np.zeros((len(input_texts),max_decoder_seq_length),dtype='float32')
decoder_target_data = np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')
 
 
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    
    for t,char in enumerate(input_text):
       encoder_input_data[i,t] = input_token_index[char]
 
    encoder_input_data[i,t+1:] = input_token_index[' ']
 
    for t,char in enumerate(target_text):
        decoder_input_data[i,t] = target_token_index[char]
        if(t>0):
          decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    
    decoder_input_data[i,t+1:] = target_token_index[' ']
    decoder_target_data[i,t:,target_token_index[' ']] = 1.0

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())